# Importando bibliotecas necessarias para para modelagem

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import joblib

# Leitura dos dados

In [8]:
df=pd.read_csv('diamonds_workshop.csv')


# Passo a passo da modelagem

In [9]:
def create_log_columns(df,param):
    #como na coluna rick não temos a coluna price, foi criado parametro que define se ela sera criada ou não
    if param == 1:
        df['price_log'] = np.log(df['price'])
    #para as demais foi feito o log sendo que nas colunas que possam ter zeros é feito log(x+1)
    df['carat_log']=np.log(df['carat'])
    df = df[['carat_log','price_log','cut_num','color_num','clarity_num']]
    return df

In [10]:
def ger_model(df,modelo):
    #cria uma função que gera um modelo com base no dataframe inputa
    y= df['price_log']
    X= df[['carat_log','cut_num','color_num']]
    modelo.fit(X, y)
    return modelo

In [11]:
def create_models_df(div_lst,modelos):
    df_models = pd.DataFrame(zip(div_lst,modelos),columns=['clarity_num','model_name'])
    df_models.apply(lambda row: joblib.dump(row['model_name'],str(row['clarity_num'])+'.pkl'),axis=1)
    df_models['model_name'] = df_models['clarity_num'].apply(lambda x: str(x)+'.pkl')
    return df_models

In [12]:
def do_predict(row,df_models):
    #para cada linha é buscado o modelo correspondente e feito o predict
    #busca qual o clarity na linha
    clar = row['clarity_num']
    #define o input conforme os dados da linha
    X= [[row['carat_log'],row['cut_num'],row['color_num']]]
    #busca o modelo correspondente conforme a clarity
    model = df_models.loc[df_models['clarity_num']==clar,'loaded_model'].values[0]
    #realiza a previsão
    y_predict = model.predict(X)
    #faz o exponencial para voltar a conversão feita no log
    y_predict=np.exp(y_predict)
    #retorna y_predict
    return y_predict

# Execução do passo a passo


In [13]:
#aplica a função que trata o dataframe
df_tratado = create_log_columns(df,1)


In [14]:
df.head()

,carat,depth,table,price,x,y,z,clarity_num,cut_num,color_num,price_log,carat_log
0,0.23,61.5,55.0,326,3.95,3.98,2.43,2,5,6,5.786897,-1.469676
1,0.74,62.2,56.0,2761,5.80,5.84,3.62,2,5,6,7.923348,-0.301105
2,0.73,61.8,58.0,2805,5.77,5.81,3.58,2,5,6,7.939159,-0.314711
3,0.77,60.7,55.0,2834,6.01,5.95,3.63,2,5,6,7.949444,-0.261365
4,0.81,60.3,57.0,2864,6.07,6.04,3.65,2,5,6,7.959975,-0.210721


In [15]:
df_tratado.head()

,carat_log,price_log,cut_num,color_num,clarity_num
0,-1.469676,5.786897,5,6,2
1,-0.301105,7.923348,5,6,2
2,-0.314711,7.939159,5,6,2
3,-0.261365,7.949444,5,6,2
4,-0.210721,7.959975,5,6,2


In [16]:
#cria uma lista com todos dados unicos na coluna clarity
clar_lst = df_tratado.clarity_num.unique()
print(clar_lst)

[2 3 5 4 6 7 1 8]


In [17]:
#cria uma lista com dataframes para cada tipo de clarity
dfs = [df_tratado.query(f'(clarity_num =={x})') for x in clar_lst]


In [18]:
# Gera modelos e salva em um dataframe
modelos = [ger_model(df,LinearRegression()) for df in dfs]
df_models = create_models_df(clar_lst,modelos)
df_models.to_csv('models.csv',index=False)
df_models['loaded_model'] = df_models['model_name'].apply(lambda x: joblib.load(x))

In [23]:
df_models

,clarity_num,model_name,loaded_model
0,2,2.pkl,LinearRegression()
1,3,3.pkl,LinearRegression()
2,5,5.pkl,LinearRegression()
3,4,4.pkl,LinearRegression()
4,6,6.pkl,LinearRegression()
5,7,7.pkl,LinearRegression()
6,1,1.pkl,LinearRegression()
7,8,8.pkl,LinearRegression()


# Previsão e avaliação do modelo

In [26]:
#cria um array com as previsões conforme cada linha do dataframe
y_predict = df.apply(do_predict, axis=1, df_models = df_models)
#seleciona a coluna real para comparar com o previsto
y_real = df['price']
#calcula r2 do model
metrics.r2_score(y_real, y_predict).round(3)

0.963